# API KEY

In [1]:
# Simpan API Key OpenRouter kamu
OPENROUTER_API_KEY = "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"  # <-- Ganti dengan punyamu

# Library

In [2]:
import json
import os
import faiss
import numpy as np
import requests
from sentence_transformers import SentenceTransformer
from gtts import gTTS
from moviepy import TextClip, AudioFileClip, CompositeVideoClip

In [3]:
# Load FAISS Index
index = faiss.read_index("output/faiss_index.bin")

# Load Mapping sumber chunk
with open("output/chunk_mapping.json", "r", encoding="utf-8") as f:
    chunk_sources = json.load(f)

# Load ulang semua chunk text
with open("output/cleaned_text/epub_chunks_translated.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Gabungkan semua chunk
all_chunks = []
for book_title, chunks in data.items():
    for chunk in chunks:
        if chunk.strip():
            all_chunks.append(chunk)

# Load model embedding
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


In [4]:
def search_best_chunk(question, top_k=1):
    """Cari chunk paling relevan untuk pertanyaan."""
    question_embedding = model.encode([question]).astype('float32')
    D, I = index.search(question_embedding, top_k)

    results = []
    for idx in I[0]:
        if idx < len(all_chunks):
            results.append((chunk_sources[idx], all_chunks[idx]))
    return results


In [5]:
import requests

def summarize_with_deepseek(text, api_key=OPENROUTER_API_KEY):
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    prompt = f"Summarize this text into a short, clear, concise answer suitable for a 1-minute video narration:\n\n{text}"

    data = {
        "model": "tngtech/deepseek-r1t-chimera:free",  # Updated model ID
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.5
    }

    response = requests.post(url, headers=headers, json=data)

    # Check if there is an error in the response
    if response.status_code != 200:
        print(f"❌ ERROR: {response.status_code} - {response.text}")
        return "[SUMMARY FAILED]"

    response_data = response.json()

    if 'choices' not in response_data:
        print(f"❌ ERROR: {response_data}")
        return "[SUMMARY FAILED]"

    summary = response_data['choices'][0]['message']['content']
    return summary


In [6]:
def text_to_speech(text, output_audio_path="output/answer_audio.mp3"):
    tts = gTTS(text=text, lang='en')
    tts.save(output_audio_path)
    return output_audio_path


In [7]:
def create_video(text, audio_path, output_video_path="output/answer_video.mp4"):
    # Buat TextClip dengan parameter yang benar
    text_clip = TextClip(
        text, 
        fontsize=70,
        color='white',
        size=(1280, 720),
        bg_color='black',
        font="Arial-Bold",
        method='label'
    )
    
    # Set durasi sesuai audio
    audio_clip = AudioFileClip(audio_path)
    text_clip = text_clip.set_duration(audio_clip.duration)
    text_clip = text_clip.set_audio(audio_clip)
    
    # Tulis video
    final_video = CompositeVideoClip([text_clip])
    final_video.write_videofile(
        output_video_path, 
        fps=24,
        codec='libx264',
        audio_codec='aac'
    )
    
    return output_video_path

In [8]:
# Pertanyaan user
question = "What does Islam say about the importance of youth?"

# Step 1: Cari Chunk
results = search_best_chunk(question, top_k=1)
source, answer = results[0]

print(f"📖 Source: {source}")
print(f"📝 Original Answer: {answer}")

# Step 2: Summarize dengan DeepSeek
summarized_answer = summarize_with_deepseek(answer)
print(f"📝 Summarized Answer: {summarized_answer}")

# Step 3: Buat Audio
audio_path = text_to_speech(summarized_answer)



📖 Source: الأربعون الشبابية - Chunk 1
📝 Original Answer: Cover forty youth Book: Forty youth, author: Muhammad Khair Ramadan Yusuf, unknown, forty youth, Muhammad Khair Ramadan Muharram 1438 AH. Source: The Golden Comprehensive is anonymous in the name of God, the Most Gracious, the Most Merciful. And educate them to strengthen, and direct them to advocacy and jihad, such as Ali, Musab, Moaz, Ibn Masoud, Ibn Omar, and others, may God be pleased with them all. And their news is many in the Sunnah and the generous prophetic biography, and if the word (youth) and (the boys) did not mention and what they behave from them, they were the audience of the new religion and its fuel, and the movement of society and its activity and vitality, and the Messenger of God, peace and blessings of God be upon him, assigned them to great deeds, to feel the responsibility of the religion About him with senior leaders. I loved the collection of a group of hadiths in which the young and young people are men

In [21]:
import os
import requests
from moviepy import TextClip, AudioFileClip, CompositeVideoClip, VideoFileClip, change_settings

# Configure MoviePy to use downloaded ffmpeg (helps avoid path issues)
change_settings({"FFMPEG_BINARY": "ffmpeg"})

def generate_pika_video(prompt):
    """Generate video using Pika Labs API"""
    try:
        url = "https://api.pika.art/v1/generate"
        payload = {
            "prompt": prompt,
            "negative_prompt": "low quality, blurry, text",
            "type": "text-to-video"
        }
        response = requests.post(url, json=payload, timeout=30)
        response.raise_for_status()
        return response.json()['video_url']
    except Exception as e:
        print(f"❌ Pika Labs API Error: {str(e)}")
        return None

def generate_video_complete(text, audio_path, output_path="output/final.mp4"):
    """Generate complete video with Pika background, text overlay, and audio"""
    
    # Create output directory if not exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    try:
        # Step 1: Generate Pika background video
        print("🔄 Generating background video with Pika Labs...")
        bg_prompt = f"Minimalist abstract background for educational content about: {text[:80]}"
        pika_url = generate_pika_video(bg_prompt)
        
        if not pika_url:
            raise Exception("Failed to generate Pika video background")
        
        # Step 2: Create text overlay (with improved formatting)
        print("✍️ Creating text overlay...")
        txt_clip = TextClip(
            text,
            fontsize=28,
            color='white',
            bg_color='rgba(0,0,0,0.5)',  # Semi-transparent background
            size=(760, 540),  # Smaller than video to add padding
            font='Arial-Bold',
            method='caption',
            align='center'
        )
        
        # Set duration from audio
        audio_clip = AudioFileClip(audio_path)
        txt_clip = txt_clip.set_duration(audio_clip.duration)
        
        # Step 3: Combine all elements
        print("🎬 Combining video elements...")
        bg_clip = VideoFileClip(pika_url).resize(height=720).set_duration(audio_clip.duration)
        
        final = CompositeVideoClip([
            bg_clip,
            txt_clip.set_position(('center'))
        ]).set_audio(audio_clip)
        
        # Optimized video export
        print("💾 Saving final video...")
        final.write_videofile(
            output_path,
            fps=24,
            codec='libx264',
            audio_codec='aac',
            threads=4,
            preset='fast',
            bitrate='3000k'
        )
        
        print(f"✅ Video successfully created at: {output_path}")
        return output_path
        
    except Exception as e:
        print(f"❌ Video generation failed: {str(e)}")
        return None

# Usage example
video_path = generate_video_complete(
    summarized_answer, 
    "output/answer_audio.mp3",
    "output/final_video.mp4"
)

if video_path:
    print(f"🎥 Final video saved at: {video_path}")
else:
    print("Failed to generate video")

ImportError: cannot import name 'change_settings' from 'moviepy' (c:\Users\Leviathans\AppData\Local\Programs\Python\Python311\Lib\site-packages\moviepy\__init__.py)

In [15]:
!pip show moviepy

Name: moviepy
Version: 2.1.2
Summary: Video editing with Python
Home-page: 
Author: Zulko 2024
Author-email: 
License: MIT License
Location: C:\Users\Leviathans\AppData\Local\Programs\Python\Python311\Lib\site-packages
Requires: decorator, imageio, imageio_ffmpeg, numpy, pillow, proglog, python-dotenv
Required-by: 
